# Resample Fruad=1 Class
- Train using Keras DNN

In [1]:
import pandas as pd
import numpy as np
import keras

from collections import Counter

from sklearn.utils import resample, shuffle

from keras.layers import Dense, Dropout, BatchNormalization

from sklearn.metrics import roc_auc_score

from multiprocessing import cpu_count

from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTE

import xgboost as xgb

!python -c 'import tensorflow as tf; print(tf.version)'
print(cpu_count())

Using TensorFlow backend.


<module 'tensorflow._api.v1.version' from '/home/ec2-user/anaconda3/lib/python3.7/site-packages/tensorflow/_api/v1/version/__init__.py'>
4


# Read Dataset

In [8]:
def get_string_features(df):
    string_features = []
    for col in df.columns:
        if df[col].dtype == np.dtype('object'):
            string_features.append(col)

    return string_features


def read_data(filename='./datasets/kfold/transaction_fold_0_0_0.csv',
              filename_identity=None,
              drop_string_features=True):
    
    df = pd.read_csv(filename)
    
    if filename_identity is not None:
        df_identity = pd.read_csv(filename_identity)
        
        df = pd.merge(df, df_identity, on='TransactionID', how='left')
        
        del df_identity
    
    
    if drop_string_features:
        string_features = get_string_features(df)
        df = df.drop(columns=string_features)
    
    return df

In [9]:
filename = './datasets/train_transaction.csv'
filename_identity = './datasets/train_identity.csv'
%time train = read_data(filename, filename_identity, drop_string_features=True)

CPU times: user 27.6 s, sys: 9.91 s, total: 37.6 s
Wall time: 35.2 s


In [10]:
train.shape

(590540, 403)

In [13]:
train.isFraud.value_counts()

0    569877
1     20663
Name: isFraud, dtype: int64

# Undersample majority class

In [14]:
def preprocess(df):
    df = df.fillna(-999)
    
    return df.drop(columns=['TransactionID', 'TransactionDT'])


train = preprocess(train)

In [15]:
train.shape

(590540, 401)

In [16]:
def split_features_labels(df):
    return df.drop(columns=['isFraud']), df['isFraud']

In [ ]:
# def undersample_then_oversample(df, random_state=27):
#     """
#     Upsample minority class (isFraud=1), combine with majority class, and then shuffle them.
#     """
#     sampling_strategy = 0.5
# #     X = df.drop(columns=['isFraud'])
# #     y = df['isFraud']
#     X, y = split_features_labels(df)
    
#     # Declare Random Under Sampler
#     rus = NearMiss(version=3, 
#                    sampling_strategy=sampling_strategy,
#                    n_jobs=cpu_count(),
#                    random_state=random_state)
    
#     ros = SMOTE(n_jobs=cpu_count(),
#                random_state=random_state)
    
#     X, y = rus.fit_resample(X, y)
#     X, y = ros.fit_resample(X, y)
    
#     return X, y.astype('int32')


# %time X, y = undersample_then_oversample(train)

In [17]:
def oversample(df, random_state=27):
    """
    Upsample minority class (isFraud=1), combine with majority class, and then shuffle them.
    """
    sampling_strategy = 0.15
#     X = df.drop(columns=['isFraud'])
#     y = df['isFraud']
    X, y = split_features_labels(df)
    
    ros = SMOTE(sampling_strategy=sampling_strategy,
                n_jobs=cpu_count(),
               random_state=random_state)
    
    X, y = ros.fit_resample(X, y)
    
    return X, y.astype('int32')


%time X, y = oversample(train)

CPU times: user 43.3 s, sys: 4.5 s, total: 47.8 s
Wall time: 17.5 s


In [18]:
print(X.shape)
print(y.shape)

(655358, 400)
(655358,)


In [19]:
Counter(y)

Counter({0: 569877, 1: 85481})

# Train using XGB

In [20]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590540 entries, 0 to 590539
Columns: 401 entries, isFraud to id_32
dtypes: float64(399), int64(2)
memory usage: 1.8 GB


In [21]:
del train

import gc
gc.collect()

630

In [22]:
seed = 27
model = xgb.XGBClassifier(objective='binary:logistic',
                            n_jobs=cpu_count(),
                            seed=seed)

%time model.fit(X, y, verbose=True)

CPU times: user 14min 1s, sys: 4.63 s, total: 14min 5s
Wall time: 3min 36s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=4,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27,
              silent=None, subsample=1, verbosity=1)

In [23]:
pred_prob = model.predict_proba(X)

score = roc_auc_score(y, pred_prob[:, 1])

print('roc-auc score={}'.format(score))

roc-auc score=0.9517415600285447


# Save Model

In [24]:
import pickle

filename_model = './models/xgboost/transaction_identity_oversampling.pkl'

with open(filename_model, 'wb') as f:
    pickle.dump(model, f)

# Predict Test Data And Write to File

In [33]:
filename = './datasets/test_transaction.csv'
filename_identity = './datasets/test_identity.csv'
%time test = read_data(filename, filename_identity, drop_string_features=True)

In [34]:
test_id = test['TransactionID']

In [35]:
test.shape

(506691, 402)

In [36]:
test = preprocess(test)

In [37]:
test.head()

,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,dist2,C1,...,id_17,id_18,id_19,id_20,id_21,id_22,id_24,id_25,id_26,id_32
0,31.95,10409,111.0,150.0,226.0,170.0,87.0,1.0,-999.0,6.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
1,49.00,4272,111.0,150.0,226.0,299.0,87.0,4.0,-999.0,3.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2,171.00,4476,574.0,150.0,226.0,472.0,87.0,2635.0,-999.0,2.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
3,284.95,10989,360.0,150.0,166.0,205.0,87.0,17.0,-999.0,5.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
4,67.95,18018,452.0,150.0,117.0,264.0,87.0,6.0,-999.0,6.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [38]:
del X
import gc
gc.collect()

NameError: name 'X' is not defined

In [39]:
%time pred_prob = model.predict_proba(test.values)

CPU times: user 6.14 s, sys: 2.62 s, total: 8.76 s
Wall time: 4.72 s


In [40]:
test_result_df = pd.DataFrame(test_id, columns=['TransactionID'])
test_result_df['isFraud'] = pred_prob[:, 1]

filename_prediction = './prediction_test_oversampling_transaction_identity.csv'
test_result_df.to_csv(filename_prediction, index=False)